# Fashion Master Category Classification using Fast AI(99% accuracy)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
# importing fast ai modules
from fastai.vision import *
from fastai.metrics import error_rate,accuracy

# Downloading the data outside kaggle kernel


1. !pip install kaggle
2. !mkdir ~/.kaggle
3. ! cp kaggle.json ~/.kaggle
4. ! chmod 600 ~/.kaggle/kaggle.json
5. !kaggle datasets download paramaggarwal/fashion-product-images-small --unzip



# Creating Data Bunch File

In [ ]:
#reading the styles.csv file
df = pd.read_csv('/kaggle/input/fashion-product-images-small/styles.csv', header = 'infer',error_bad_lines = False)
df = df[['id','masterCategory']] # filtering to 2 columns of our interest


In [ ]:
# additional step to remove the files which is there in df but not there in the images folder
files=glob.glob("/kaggle/input/fashion-product-images-small/images/*")
files[2]

In [ ]:
img = []
for file in files:
    file = file.replace('/kaggle/input/fashion-product-images-small/images/','').replace('.jpg','')
    img.append(file)

diff = set(df['id'].astype(str)) - set(img)

In [ ]:
diff

In [ ]:
diff = list(map(int,list(diff)))
df = df[~df.id.isin(diff)]

In [ ]:
df.shape

In [ ]:
df.to_csv('/kaggle/working/styles_cleaned.csv')

# Creating the data to run resnet 34

In [ ]:
tfms = get_transforms(do_flip=False)

In [ ]:
bs = 64 #batch size

In [ ]:
os.chdir('/kaggle/input/fashion-product-images-small/')

In [ ]:
data = (ImageList.from_csv('', '/kaggle/working/styles_cleaned.csv', folder='images', suffix='.jpg', cols = 'id')
        #Where to find the data? -> in planet 'train' folder
        .split_by_rand_pct()
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_df(cols = 'masterCategory')
        #How to label? -> use the second clumn of the csv file and split the tags by ' '
        .transform(tfms, size=128)
        #Data augmentation? -> use tfms with a size of 128
        .databunch(bs = bs))                        
        #Finally -> use the defaults for conversion to databunch

In [ ]:
data.show_batch(rows=5, figsize=(4,4))

In [ ]:
data.classes

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate,accuracy])
learn.fit_one_cycle(4)# results below

#epoch	train_loss	valid_loss	error_rate	time
0	0.211633	0.118356	0.030620	01:53
1	0.128470	0.072357	0.016999	00:40
2	0.080400	0.055352	0.012608	00:40
3	0.067283	0.051523	0.012721	00:40